In [3]:
import xmltodict
import pandas as pd
import numpy as np
import json
import os

In [4]:
os.getcwd()
os.chdir('D:\\jeiner\\jeinercastro\\datascience\\laboratorio1\\data')

In [4]:
boletas = 'D:\\jeiner\\jeinercastro\\datascience\\laboratorio1\\data\\boletas_mayo_oea.json'
facturas = 'D:\\jeiner\\jeinercastro\\datascience\\laboratorio1\\data\\facturas_mayo.json'

In [2]:
f = open( boletas , "rb" )
jsonObjectBoletas = json.load(f)


NameError: name 'boletas' is not defined

In [1]:
data = []
for item in jsonObjectBoletas:
    jsonload = json.loads(item['bol_response'])
    dictionary = xmltodict.parse(jsonload['xml'])
    print(dictionary)
    break
    total = str(dictionary['Invoice']['cac:InvoiceLine']['cac:PricingReference']['cac:AlternativeConditionPrice']['cbc:PriceAmount']['#text'])
    total = int(total)
    sutotal = round(total / (1 + 0.18), 2)

    igv = round(total - sutotal, 2)
    sutotal = total - igv

    num_correlativo = item['bol_correlativo']
    dni = dictionary['Invoice']['cac:AccountingCustomerParty']['cac:Party']['cac:PartyIdentification']['cbc:ID']['#text']
    tipo_doc = dictionary['Invoice']['cac:AccountingCustomerParty']['cac:Party']['cac:PartyIdentification']['cbc:ID']['@schemeID']

    descripcion = dictionary['Invoice']['cac:InvoiceLine']['cac:Item']['cbc:Description']
    course_code =  dictionary['Invoice']['cac:InvoiceLine']['cac:Item']['cac:SellersItemIdentification']['cbc:ID']

    leyenda = dictionary['Invoice']['cbc:Note']['#text']
    nombres = dictionary['Invoice']['cac:AccountingCustomerParty']['cac:Party']['cac:PartyLegalEntity']['cbc:RegistrationName']

    description_sunat = ''
    if jsonload['sunatResponse'].get('cdrResponse', None) is not None:
        description_sunat = jsonload['sunatResponse']['cdrResponse']['description']
    elif jsonload['sunatResponse'].get('error'):
        description_sunat = jsonload['sunatResponse']['error']['message']
    obj = {
        'diplomado_code': course_code,
        'cliente': nombres,
        'dni': dni,
        'serie': 'B001',
        'correlativo': num_correlativo,
        'total': total,
        'subtotal': sutotal,
        'igv': igv,
        'sunat_description': description_sunat
    }
    data.append(obj)




NameError: name 'jsonObjectBoletas' is not defined

In [32]:
df = pd.DataFrame(data)
df.head()

,diplomado_code,cliente,dni,serie,correlativo,total,subtotal,igv,sunat_description
0,CAA0096,ROSA YSABEL MESONES ZUÑIGA,41658603,B001,12298,150,127.12,22.88,"La Boleta de Venta numero B001-12298, ha sido ..."
1,CAA0099,ALEXANDRA ROXANA RODRIGUEZ RAMOS,44628593,B001,12299,150,127.12,22.88,"La Boleta de Venta numero B001-12299, ha sido ..."
2,CAA0107,OLINDA AZUCENA YGLESIAS CUENCA,15354786,B001,12300,150,127.12,22.88,"La Boleta de Venta numero B001-12300, ha sido ..."
3,CAA0114,MAGALY MAGDALENA MENDOZA JULCA,44186427,B001,12301,170,144.07,25.93,"La Boleta de Venta numero B001-12301, ha sido ..."
4,CAA0117,SOLEDAD MARGARITA ESPINOZA ARTEAGA,45155433,B001,12302,50,42.37,7.63,"La Boleta de Venta numero B001-12302, ha sido ..."


In [33]:
df.to_csv('D:\\jeiner\\jeinercastro\\datascience\\laboratorio1\\data\\boletas_3_mayo_oea.csv')
